In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from typing import Dict, Any

# Assuming your model class is defined like this:
# You'll need to import your actual model class
class CreditRiskModel:
    def __init__(self, model_path: str):
        # Initialize your model
        # This is a placeholder - replace with your actual implementation
        self.model_path = model_path
        # Here you would load your model, scaler, etc.
        
    def predict(self, client_data: Dict[str, Any]) -> int:
        # Your prediction logic
        # This is a placeholder - replace with your actual implementation
        return 0  # or 1

# Create a Pydantic model for request validation
class ClientData(BaseModel):
    # Define the expected client data fields
    # Example fields - replace with your actual required fields
    age: int
    income: float
    credit_score: int
    loan_amount: float
    loan_term: int
    # Add more fields as needed based on your model's requirements

# Create a Pydantic model for the response
class PredictionResponse(BaseModel):
    will_repay: int  # 0 or 1
    request_id: str  # To track requests

# Initialize the FastAPI application
app = FastAPI(
    title="Credit Risk Prediction API",
    description="API for predicting if a client will repay a credit",
    version="0.1.0"
)

# Initialize your model
# Update the path to where your model is stored
#model = CreditRiskModel(model_path="path/to/your/model")

model = "path/to/your/model"

@app.post("/predict", response_model=PredictionResponse)
async def predict_credit_risk(client_data: ClientData):
    try:
        # Convert Pydantic model to dictionary
        client_dict = client_data.dict()
        
        # Get prediction from your model
        prediction_proba = model.predict_proba(client_dict)

        prediction = model.predict(client_dict)
        
        # Create a unique request ID (you may want to use UUID or another method)
        import time
        request_id = f"req_{int(time.time())}"
        
        # Return the prediction
        return PredictionResponse(
            proba=prediction_proba
            will_repay=prediction,
            request_id=request_id
        )
    except Exception as e:
        # Handle exceptions
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")

# Health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "healthy"}

# Root endpoint with API info
@app.get("/")
async def root():
    return {
        "api": "Credit Risk Prediction API",
        "version": "0.1.0",
        "endpoints": {
            "/predict": "POST - Get credit risk prediction",
            "/health": "GET - API health check"
        }
    }

# For running the application locally
if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)